# RAG for Product Category Classification

1. Category Encoding
2. Embedding Creation
3. Top 5 Retrieval.
4. Accuracy Assessment
5. Prompt Accuracy



## Define Function


In [289]:
#authorize
import hmac
import urllib
import requests
import arrow
import base64
import string
import random
import hashlib
import json
from typing import Dict
import pandas as pd
import numpy as np
import concurrent.futures
import re
import time
import ast

import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity



MAX_RETRIES = 10



pd.set_option('max_colwidth', 400)

import warnings
warnings.filterwarnings('ignore')




### Batch Request Embedding

In [285]:



def embedding_request(text):
    url = 'xxxx'

    headers = {
        'xxx': 'xxx',
    }


    pay_load = {
        "bizId":"embedding-3",
        "model": "text-embedding-3-small",
        "input": text

    }

    retries = 0
    while retries <= MAX_RETRIES:
        try:
            
            response = requests.post(url, json=pay_load, headers=headers)
            if response.status_code == 200:
                try:
                    return response.json()['data'][0]['embedding']
                except Exception as e:
                    #print(f'【返回异常】请求成功 json解析异常{response.json()}.\n')
                    return None

            else:
                print(f"【状态码异常】Error: the API responded with status code {response.status_code}.\n")
                return None
        
        except requests.Timeout:
            retries += 1
            print(f"【请求超时】Retry {retries}/{MAX_RETRIES}")
    print(f"【请求失败】Reached maximum retries: {MAX_RETRIES}")
    return None
 



# 调用接口并更新DataFrame的函数
def update_df_with_api_response(row_index, text):
    
    response = embedding_request( text)
    # 返回结果以便之后更新DataFrame
    return row_index, response

def check_embedding_exists(embedding):
    """Check if the embedding exists and is not empty."""
    if isinstance(embedding, np.ndarray):
        return embedding.size > 0
    return False


# 使用ThreadPoolExecutor批量调用接口
def dataframe_request_embed_notna(df,text_col,num_threads):
    """
    params : {"df":"dataframe",
    "text_col":"text need to be embedded from the dataframe above",
    "num_threads":"thread number"}
    """
    
    with concurrent.futures.ThreadPoolExecutor(max_workers = num_threads) as executor:
        futures = []
        for index, row in df.iterrows():
            # 检查api_response列是否为空，如果为空则调用接口
            if not check_embedding_exists(row['api_response']):
                futures.append(executor.submit(update_df_with_api_response, index, row[text_col] ))

        # 等待所有线程任务完成，并收集结果
        for future in concurrent.futures.as_completed(futures):
            row_index, response = future.result()
            # 更新DataFrame
            df.at[row_index, 'api_response'] = response



### Batch Request GPT

In [283]:


def request_gpt( text, prompt,temp, model):
    text = '"""'+text+'"""'
    pay_load = {
        "temperature":temp,
        "messages": [
            {"role":"system","content":prompt},
            {"role":"user", "content":text}
        ],
        "model":model
    }

    retries = 0
    while retries <= MAX_RETRIES:
        try:
            
            response = requests.post(url, json=pay_load, headers = AuthUtil.generate_request_headers(request_path), timeout=10)
            if response.status_code == 200:
                try:
                    return response.json()['choices'][0]['message']['content']
                except Exception as e:
                    #print(f'【返回异常】请求成功 json解析异常{response.json()}.\n')
                    return None

            else:
                print(f"【状态码异常】Error: the API responded with status code {response.status_code}.\n")
                return None
        
        except requests.Timeout:
            retries += 1
            print(f"【请求超时】Retry {retries}/{MAX_RETRIES}")
    print(f"【请求失败】Reached maximum retries: {MAX_RETRIES}")
    return None




# 调用接口并更新DataFrame的函数
def update_df_with_api_response(row_index, text, prompt, temp,model):

    response = request_gpt( text,prompt, temp,model)
    # 返回结果以便之后更新DataFrame
    return row_index, response


def dataframe_request_notna_gpt35(df, text_col, prompt, temp=0.2, num_threads=3, model='chat'):
    """
    Process the DataFrame to make API requests for non-empty, non-None entries in a specified column.

    Params:
    - df (pd.DataFrame): The DataFrame to process.
    - text_col (str): The column from which text needs to be analyzed.
    - prompt (str): System prompt for the API request.
    - temp (float, optional): Temperature setting for the API model, default is 0.2.
    - num_threads (int, optional): Number of concurrent threads to use, default is 3.
    - model (str, optional): Model identifier for the API, default is 'chat'.
    """
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = []
        for index, row in df.iterrows():
            # Check if the text column is empty, NaN or None, skip these rows
            if pd.isna(row[text_col]) or row[text_col] == '' or row[text_col] is None:
                continue
            # Check if api_response is empty or NaN, then initiate an API call
            if pd.isna(row['api_response']) or row['api_response'] == '':
                futures.append(executor.submit(update_df_with_api_response, index, row[text_col], prompt, temp, model))

        # Wait for all thread tasks to complete and collect results
        for future in concurrent.futures.as_completed(futures):
            row_index, response = future.result()
            # Update DataFrame
            df.at[row_index, 'api_response'] = response

# Example usage:
# Assume df is your DataFrame, 'text_col' is the column name containing text, and 'api_response' is the column to be updated




### Other Functions

In [9]:



# Function to clean JSON strings, if needed
def clean_json(json_str):
    # Check if the JSON string is not null
    if pd.notna(json_str):
        # Find the first occurrence of a valid JSON structure
        match = re.search(r'{.*}', json_str, re.DOTALL)
        if match:
            cleaned_json_str = match.group(0)
            # Attempt to load the JSON to see if it's valid
            try:
                # This also helps in removing unnecessary whitespaces
                cleaned_json = json.loads(cleaned_json_str)
                # Convert it back to a string to ensure consistency
                return json.dumps(cleaned_json)
            except json.JSONDecodeError:
                pass  # If JSON is invalid, you might want to handle this case.
    return ''


# Extract key from JSON string
def extract_category_type(json_str, entity):
    try:
        # Load the JSON string into a dictionary
        data = json.loads(json_str)
        # Extract the value for the specified entity
        return data.get(entity, None)
    except json.JSONDecodeError:
        # Return None if there is an error decoding the JSON
        return None

# Extract keys from DataFrame column containing JSON strings
def df_extract_category_type(entity_list, df, extracted_col):
    # Ensure the JSON strings are clean and proper; this step is optional based on your data quality
    df[extracted_col] = df[extracted_col].apply(clean_json)
    
    # For each entity in the list, create a new column in the DataFrame based on the extracted data
    for entity in entity_list:
        # Apply the extraction function across the DataFrame column
        df[entity] = df[extracted_col].apply(lambda x: extract_category_type(x, entity))
    
    return df
    
    
#sample based on certain column
#抽样逻辑，需要对每个站点随机取1k

def sample_df(df_samp,samp_col,sample_size ):

    df_samped = pd.DataFrame()
    for mall in df_samp[samp_col].unique():
        mall_spus = df_samp[df_samp[samp_col] == mall]

        if len(mall_spus) >= sample_size:
            mall_spus = mall_spus.sample(sample_size,replace = False)

        else:
            mall_spus = mall_spus.sample(len(mall_spus),replace = False)

        df_samped = df_samped.append(mall_spus,ignore_index=True)    
        
        
    return df_samped



def sample_dataframe(df, sample_column, sample_size):
    """
    Samples a specified number of rows from each group in a DataFrame,
    defined by unique values in a specified column.

    Parameters:
        df (pd.DataFrame): The DataFrame to sample from.
        sample_column (str): The column name to group by and sample within each group.
        sample_size (int): The number of samples to attempt to take from each group.

    Returns:
        pd.DataFrame: A DataFrame containing the sampled rows.
    """

    sampled_dfs = []  # List to store sampled DataFrames for each group.

    for value in df[sample_column].unique():
        group_df = df[df[sample_column] == value]
        actual_sample_size = min(len(group_df), sample_size)  # Take the minimum of the group size or the desired sample size.
        sampled_group = group_df.sample(n=actual_sample_size, replace=False)  # Sample without replacement.
        sampled_dfs.append(sampled_group)

    return pd.concat(sampled_dfs, ignore_index=True)  # Concatenate all sampled group DataFrames.



## Load Data


sampling product data 

`select img_mid_url,goods_name_en
,sku_cate_1_nm,sku_cate_2_nm,sku_cate_3_nm,sku_cate_4_nm,cate_last,rn  from (
select img_mid_url,goods_name_en
,json_extract_scalar(cate_l1, '$.en') AS sku_cate_1_nm
,json_extract_scalar(cate_l2, '$.en') AS sku_cate_2_nm
,json_extract_scalar(cate_l3, '$.en') AS sku_cate_3_nm
,json_extract_scalar(cate_l4, '$.en') AS sku_cate_4_nm
,json_extract_scalar(cate_last, '$.en') AS cate_last
,ROW_NUMBER() OVER( partition by cate_last ORDER BY matrix_xxx DESC) AS rn
from xxx_table
where site_tp = 'shein'
and area_nm = 'United States'
)
where rn<=40
and sku_cate_1_nm in ('Home & Living','Women Apparel')`

In [134]:
df_category = pd.read_excel('./category_0322.xlsx')[['sku_cate_nm','sku_cate_1_nm','sku_cate_2_nm','sku_cate_3_nm','sku_cate_4_nm']].drop_duplicates()

In [135]:
df_category.head(3)

,sku_cate_nm,sku_cate_1_nm,sku_cate_2_nm,sku_cate_3_nm,sku_cate_4_nm
0,Women Skirts,Women Apparel,Women Clothing,Women Bottoms,Women Skirts
1,Women Blouses,Women Apparel,Women Clothing,"Women Tops, Blouses & Tee",Women Blouses
2,Women Coats,Women Apparel,Women Clothing,Women Outerwear,Women Coats


In [120]:
df_prod= pd.read_excel('./df_prod_dup_filt_concat_result_0403.xlsx')

In [121]:
df_prod[['goods_name_en','sku_cate_1_nm','sku_cate_2_nm','sku_cate_3_nm','sku_cate_4_nm']].head(3)

,goods_name_en,sku_cate_1_nm,sku_cate_2_nm,sku_cate_3_nm,sku_cate_4_nm
0,"36-Pack Cedar Rings For Plastic Or Wire Hangers, Natural Cedar Wood Blocks For Closets, Cabinets, Drawers, Shoes, Clothes Storage Freshener, DIY Crafts (2x2 In)",Home & Living,Household Merchandises,Moth & Mildew Proofing,Active Carbon Bags & Boxes
1,"1/3/5pcs Refrigerator Deodorant, Natural Activated Carbon Refrigerator Deodorizer, Long-Lasting Microwave Oven Cleaner, More Effective Than Baking Soda, Kitchen Cleaning Tool For Refrigerator, Cooler, Microwave Oven, And Lunch Box In Restaurants",Home & Living,Household Merchandises,Moth & Mildew Proofing,Active Carbon Bags & Boxes
2,"Household Refrigerator Bamboo Charcoal Box, Creative Cloud Shaped Design, Suction Type, Activated Carbon Deodorizing Box, Removes Bacteria & Odors",Home & Living,Household Merchandises,Moth & Mildew Proofing,Active Carbon Bags & Boxes


## Preprocessing Data

In [88]:
df_prod_all['sku_cate_1_nm'].value_counts()

sku_cate_1_nm
Home & Living    22187
Women Apparel     3204
Name: count, dtype: int64

In [297]:
df_category[['sku_cate_1_nm', 'sku_cate_2_nm']].value_counts()

sku_cate_1_nm             sku_cate_2_nm            
Home & Living             Kitchen & Dining             372
                          Home Decor                   185
                          Arts,Crafts & Sewing         181
                          Storage & Organization       174
                          Outdoor & Garden             134
                                                      ... 
Sports & Outdoor          Toddler Girl Activewear        1
Underwear & Sleepwear     Women Basic Tops               1
                          Nightwear                      1
                          Men Underwear Accessories      1
Tools & Home Improvement  Tool Parts                     1
Name: count, Length: 252, dtype: int64

## Embedding

### Document: hierarchical category

In [138]:
#embedding the hierarchical category structure.
df_category['json_column'] = df_category.apply(
    lambda row: json.dumps({key: value for key, value in row.items() if pd.notna(value)}),
    axis=1
)

df_category['api_response'] = ''
df_category.head(3)

,sku_cate_nm,sku_cate_1_nm,sku_cate_2_nm,sku_cate_3_nm,sku_cate_4_nm,json_column,api_response
0,Women Skirts,Women Apparel,Women Clothing,Women Bottoms,Women Skirts,"{""sku_cate_nm"": ""Women Skirts"", ""sku_cate_1_nm"": ""Women Apparel"", ""sku_cate_2_nm"": ""Women Clothing"", ""sku_cate_3_nm"": ""Women Bottoms"", ""sku_cate_4_nm"": ""Women Skirts""}",
1,Women Blouses,Women Apparel,Women Clothing,"Women Tops, Blouses & Tee",Women Blouses,"{""sku_cate_nm"": ""Women Blouses"", ""sku_cate_1_nm"": ""Women Apparel"", ""sku_cate_2_nm"": ""Women Clothing"", ""sku_cate_3_nm"": ""Women Tops, Blouses & Tee"", ""sku_cate_4_nm"": ""Women Blouses""}",
2,Women Coats,Women Apparel,Women Clothing,Women Outerwear,Women Coats,"{""sku_cate_nm"": ""Women Coats"", ""sku_cate_1_nm"": ""Women Apparel"", ""sku_cate_2_nm"": ""Women Clothing"", ""sku_cate_3_nm"": ""Women Outerwear"", ""sku_cate_4_nm"": ""Women Coats""}",


In [212]:
# execuate embedding 

dataframe_request_embed_notna(df_category,'json_column',6)

#df_category = pd.read_excel('./df_category_0322_embedding.xlsx')
df_category.head(3)

,Unnamed: 0,sku_cate_1_nm,sku_cate_2_nm,sku_cate_3_nm,sku_cate_4_nm,json_column,api_response
0,0,Women Apparel,Women Clothing,Women Bottoms,Women Skirts,"{""sku_cate_1_nm"": ""Women Apparel"", ""sku_cate_2_nm"": ""Women Clothing"", ""sku_cate_3_nm"": ""Women Bottoms"", ""sku_cate_4_nm"": ""Women Skirts""}","[0.020317545, 0.03978173, 0.008251258, -0.022827433, -0.028612727, -0.031850483, -0.0006051184, 0.027106794, 0.015598956, -0.028186046, -0.021246204, -0.030168857, -0.023881588, -0.015197374, 0.0040785684, 0.0629982, 0.027106794, 0.025952246, -0.00127769, 0.06129147, 0.03353211, -0.0023185094, -0.0073853466, -0.024182774, 0.00533665, 0.0048974194, -0.014306364, 0.041839838, 0.008671664, -0.021..."
1,1,Women Apparel,Women Clothing,"Women Tops, Blouses & Tee",Women Blouses,"{""sku_cate_1_nm"": ""Women Apparel"", ""sku_cate_2_nm"": ""Women Clothing"", ""sku_cate_3_nm"": ""Women Tops, Blouses & Tee"", ""sku_cate_4_nm"": ""Women Blouses""}","[0.014059782, 0.032025732, -0.0063566226, -0.026760897, -0.021059345, -0.032850638, 0.020295095, 0.009838208, 0.010177875, -0.04539405, -0.0038758416, -0.013756508, -0.026954992, -0.027755635, 0.017844642, 0.059004985, 0.018281356, 0.026518278, 0.010226399, 0.06589537, 0.03716926, 0.0064718667, 0.0018696841, -0.0025763125, 0.018038737, 0.013040781, -0.023849467, 0.02969659, 0.0041366573, -0.00..."
2,2,Women Apparel,Women Clothing,Women Outerwear,Women Coats,"{""sku_cate_1_nm"": ""Women Apparel"", ""sku_cate_2_nm"": ""Women Clothing"", ""sku_cate_3_nm"": ""Women Outerwear"", ""sku_cate_4_nm"": ""Women Coats""}","[0.009404603, 0.024876693, 0.00086059247, -0.016221238, -0.02113714, -0.029322056, -0.010735736, 0.0155649595, 0.004767311, -0.04655867, -0.009714169, -0.011707772, -0.030535553, -0.02528532, -0.005946756, 0.052403267, 0.01999794, 0.03283872, -0.0040305443, 0.06339904, 0.0171747, 0.0008536272, 0.0009209577, -0.02075328, 0.013856158, 0.025248172, -0.00740481, 0.062259838, 0.016617483, -0.028133..."


### Question: producu title

In [132]:
#dataframe_request_embed_notna(df_prod,'goods_name_en',6)
df_prod[['goods_name_en','sku_cate_1_nm','sku_cate_2_nm','sku_cate_3_nm','sku_cate_4_nm','api_response']].head(3)

,goods_name_en,sku_cate_1_nm,sku_cate_2_nm,sku_cate_3_nm,sku_cate_4_nm,api_response
0,"36-Pack Cedar Rings For Plastic Or Wire Hangers, Natural Cedar Wood Blocks For Closets, Cabinets, Drawers, Shoes, Clothes Storage Freshener, DIY Crafts (2x2 In)",Home & Living,Household Merchandises,Moth & Mildew Proofing,Active Carbon Bags & Boxes,"[0.023559626, 0.012778686, -0.0045178914, 0.03827865, -0.003366891, -0.023295557, 0.036074243, -0.01669381, -0.016556034, -0.007835987, 0.027417343, -0.014719025, -0.00636064, -0.057590198, 0.026361063, 0.012572023, 0.042457845, 0.015029021, -0.030723957, 0.055064313, -0.002145568, 0.051665846, 0.01458125, -0.011871664, 0.021045221, 0.04638445, 0.041654155, -0.015568642, 0.0047216844, -0.00841..."
1,"1/3/5pcs Refrigerator Deodorant, Natural Activated Carbon Refrigerator Deodorizer, Long-Lasting Microwave Oven Cleaner, More Effective Than Baking Soda, Kitchen Cleaning Tool For Refrigerator, Cooler, Microwave Oven, And Lunch Box In Restaurants",Home & Living,Household Merchandises,Moth & Mildew Proofing,Active Carbon Bags & Boxes,"[0.0033222602, 0.014342376, -0.031983074, -0.006899874, -0.011139813, -0.009740686, -0.0046708477, 0.0028833707, -0.01577874, -0.013714631, 0.048006527, 0.0059476173, 0.02455653, 0.0033063006, 0.026535522, 0.03298321, 0.012022912, 0.010661024, -0.01584258, 0.026833436, 0.0049102423, 0.03242994, 0.021939153, 0.029472092, 0.021960434, 0.023811748, -0.00873523, 0.019758007, 0.023918146, -0.049198..."
2,"Household Refrigerator Bamboo Charcoal Box, Creative Cloud Shaped Design, Suction Type, Activated Carbon Deodorizing Box, Removes Bacteria & Odors",Home & Living,Household Merchandises,Moth & Mildew Proofing,Active Carbon Bags & Boxes,"[0.018331882, -0.0067473077, -0.030899731, 0.024028102, -0.028074225, -0.0003867411, -0.02251363, 0.023847269, -0.02463841, -0.018038029, 0.044936843, 0.018750057, 0.014760442, 0.01932646, 0.017970217, 0.0053232526, -0.0017885452, -0.004255212, -0.008900343, 0.030470254, -4.3398002e-05, 0.05497304, 0.052531805, 0.028594118, -0.0061483006, 0.02145124, -0.002223673, 0.00270966, -0.0011273768, -0..."


## Retriver

In [248]:
#compute similarity of product title & category classification, and return top 5


import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity #efficient cosine similarity, compared with build from scratch.


# Convert json strings in 'json_column' to dictionaries (if they are not already)
#df_prod['json_column'] = df_prod['json_column'].apply(eval)
#df_category['json_column'] = df_category['json_column'].apply(eval)

# Ensure that api_response_y and api_response are list of floats  (if they are not already)
#df_prod_test['api_response'] = df_prod['api_response'].apply(lambda x: np.array(eval(x)))
#df_category['api_response'] = df_category['api_response'].apply(eval)

# Calculate cosine similarity
# Note: The list comprehension converts the lists in api_response_y and api_response to a 2D array suitable for cosine_similarity
similarities = cosine_similarity([x for x in df_prod['api_response']], [x for x in df_category['api_response']])

# Find the indices of the top 5 most similar rows in df_b for each row in df_a
top_5_indices = [(-sim).argsort()[:50] for sim in similarities]

# Retrieve the corresponding json_column values for these indices
df_prod['top_similar_json'] = [[df_category.iloc[idx]['json_column'] for idx in indices] for indices in top_5_indices]

In [252]:
df_prod[['goods_name_en','sku_cate_1_nm','sku_cate_2_nm','sku_cate_3_nm','sku_cate_4_nm','top_similar_json']].head(3)

,goods_name_en,sku_cate_1_nm,sku_cate_2_nm,sku_cate_3_nm,sku_cate_4_nm,top_similar_json
0,"36-Pack Cedar Rings For Plastic Or Wire Hangers, Natural Cedar Wood Blocks For Closets, Cabinets, Drawers, Shoes, Clothes Storage Freshener, DIY Crafts (2x2 In)",Home & Living,Household Merchandises,Moth & Mildew Proofing,Active Carbon Bags & Boxes,"[{""sku_cate_1_nm"": ""Home & Living"", ""sku_cate_2_nm"": ""Storage & Organization"", ""sku_cate_3_nm"": ""Clothing & Closet Storage"", ""sku_cate_4_nm"": ""Underwear Storage Boxes""}, {""sku_cate_1_nm"": ""Home & Living"", ""sku_cate_2_nm"": ""Storage & Organization"", ""sku_cate_3_nm"": ""Clothing & Closet Storage"", ""sku_cate_4_nm"": ""Wardrobe Rod Holders""}, {""sku_cate_1_nm"": ""Home & Living"", ""sku_cate_2_nm"": ""Storage..."
1,"1/3/5pcs Refrigerator Deodorant, Natural Activated Carbon Refrigerator Deodorizer, Long-Lasting Microwave Oven Cleaner, More Effective Than Baking Soda, Kitchen Cleaning Tool For Refrigerator, Cooler, Microwave Oven, And Lunch Box In Restaurants",Home & Living,Household Merchandises,Moth & Mildew Proofing,Active Carbon Bags & Boxes,"[{""sku_cate_1_nm"": ""Home Appliances"", ""sku_cate_2_nm"": ""Household Appliances"", ""sku_cate_3_nm"": ""Portable Refrigerator""}, {""sku_cate_1_nm"": ""Home Appliances"", ""sku_cate_2_nm"": ""Kitchen Appliances"", ""sku_cate_3_nm"": ""Cooking Appliances"", ""sku_cate_4_nm"": ""Microwave Ovens""}, {""sku_cate_1_nm"": ""Home & Living"", ""sku_cate_2_nm"": ""Household Cleaning"", ""sku_cate_3_nm"": ""Household Chemicals"", ""sku_cat..."
2,"Household Refrigerator Bamboo Charcoal Box, Creative Cloud Shaped Design, Suction Type, Activated Carbon Deodorizing Box, Removes Bacteria & Odors",Home & Living,Household Merchandises,Moth & Mildew Proofing,Active Carbon Bags & Boxes,"[{""sku_cate_1_nm"": ""Home & Living"", ""sku_cate_2_nm"": ""Storage & Organization"", ""sku_cate_3_nm"": ""Kitchen Storage & Organization"", ""sku_cate_4_nm"": ""Refrigerator Storage Boxes""}, {""sku_cate_1_nm"": ""Home & Living"", ""sku_cate_2_nm"": ""Household Merchandises"", ""sku_cate_3_nm"": ""Moth & Mildew Proofing"", ""sku_cate_4_nm"": ""Active Carbon Bags & Boxes""}, {""sku_cate_1_nm"": ""Home Appliances"", ""sku_cate_2_..."


In [256]:
#check the candidate result
print(len(df_prod['top_similar_json'][0]))

50


## Prompt Query

In [303]:
#construct prompt to choose the most matching one from the candidate.

#df_prod = df_prod.drop(columns=['Unnamed: 0.2','Unnamed: 0.1','Unnamed: 0'])
# df_prod['product_title_embedding'] = df_prod['api_response']  #store the product title embedding result.
df_prod['input'] = 'product title:' + df_prod['goods_name_en'] + '.\n category choices:'+ df_prod['top_similar_json'].astype(str) 
df_prod['api_response'] = ''

df_prod[['goods_name_en','sku_cate_1_nm','sku_cate_2_nm','sku_cate_3_nm','sku_cate_4_nm','top_similar_json','input']].head(3)

,goods_name_en,sku_cate_1_nm,sku_cate_2_nm,sku_cate_3_nm,sku_cate_4_nm,top_similar_json,input
0,"36-Pack Cedar Rings For Plastic Or Wire Hangers, Natural Cedar Wood Blocks For Closets, Cabinets, Drawers, Shoes, Clothes Storage Freshener, DIY Crafts (2x2 In)",Home & Living,Household Merchandises,Moth & Mildew Proofing,Active Carbon Bags & Boxes,"[{""sku_cate_1_nm"": ""Home & Living"", ""sku_cate_2_nm"": ""Storage & Organization"", ""sku_cate_3_nm"": ""Clothing & Closet Storage"", ""sku_cate_4_nm"": ""Underwear Storage Boxes""}, {""sku_cate_1_nm"": ""Home & Living"", ""sku_cate_2_nm"": ""Storage & Organization"", ""sku_cate_3_nm"": ""Clothing & Closet Storage"", ""sku_cate_4_nm"": ""Wardrobe Rod Holders""}, {""sku_cate_1_nm"": ""Home & Living"", ""sku_cate_2_nm"": ""Storage...","product title:36-Pack Cedar Rings For Plastic Or Wire Hangers, Natural Cedar Wood Blocks For Closets, Cabinets, Drawers, Shoes, Clothes Storage Freshener, DIY Crafts (2x2 In).\n category choices:['{""sku_cate_1_nm"": ""Home & Living"", ""sku_cate_2_nm"": ""Storage & Organization"", ""sku_cate_3_nm"": ""Clothing & Closet Storage"", ""sku_cate_4_nm"": ""Underwear Storage Boxes""}', '{""sku_cate_1_nm"": ""Home & Li..."
1,"1/3/5pcs Refrigerator Deodorant, Natural Activated Carbon Refrigerator Deodorizer, Long-Lasting Microwave Oven Cleaner, More Effective Than Baking Soda, Kitchen Cleaning Tool For Refrigerator, Cooler, Microwave Oven, And Lunch Box In Restaurants",Home & Living,Household Merchandises,Moth & Mildew Proofing,Active Carbon Bags & Boxes,"[{""sku_cate_1_nm"": ""Home Appliances"", ""sku_cate_2_nm"": ""Household Appliances"", ""sku_cate_3_nm"": ""Portable Refrigerator""}, {""sku_cate_1_nm"": ""Home Appliances"", ""sku_cate_2_nm"": ""Kitchen Appliances"", ""sku_cate_3_nm"": ""Cooking Appliances"", ""sku_cate_4_nm"": ""Microwave Ovens""}, {""sku_cate_1_nm"": ""Home & Living"", ""sku_cate_2_nm"": ""Household Cleaning"", ""sku_cate_3_nm"": ""Household Chemicals"", ""sku_cat...","product title:1/3/5pcs Refrigerator Deodorant, Natural Activated Carbon Refrigerator Deodorizer, Long-Lasting Microwave Oven Cleaner, More Effective Than Baking Soda, Kitchen Cleaning Tool For Refrigerator, Cooler, Microwave Oven, And Lunch Box In Restaurants.\n category choices:['{""sku_cate_1_nm"": ""Home Appliances"", ""sku_cate_2_nm"": ""Household Appliances"", ""sku_cate_3_nm"": ""Portable Refrigera..."
2,"Household Refrigerator Bamboo Charcoal Box, Creative Cloud Shaped Design, Suction Type, Activated Carbon Deodorizing Box, Removes Bacteria & Odors",Home & Living,Household Merchandises,Moth & Mildew Proofing,Active Carbon Bags & Boxes,"[{""sku_cate_1_nm"": ""Home & Living"", ""sku_cate_2_nm"": ""Storage & Organization"", ""sku_cate_3_nm"": ""Kitchen Storage & Organization"", ""sku_cate_4_nm"": ""Refrigerator Storage Boxes""}, {""sku_cate_1_nm"": ""Home & Living"", ""sku_cate_2_nm"": ""Household Merchandises"", ""sku_cate_3_nm"": ""Moth & Mildew Proofing"", ""sku_cate_4_nm"": ""Active Carbon Bags & Boxes""}, {""sku_cate_1_nm"": ""Home Appliances"", ""sku_cate_2_...","product title:Household Refrigerator Bamboo Charcoal Box, Creative Cloud Shaped Design, Suction Type, Activated Carbon Deodorizing Box, Removes Bacteria & Odors.\n category choices:['{""sku_cate_1_nm"": ""Home & Living"", ""sku_cate_2_nm"": ""Storage & Organization"", ""sku_cate_3_nm"": ""Kitchen Storage & Organization"", ""sku_cate_4_nm"": ""Refrigerator Storage Boxes""}', '{""sku_cate_1_nm"": ""Home & Living"",..."


In [270]:
prompt_select_v2 = """
Your task is to choose the most appropriate category for a given product title from the provided list of category choices. 

Steps:

    1. Analyze the product title to understand the product.
    2. Review each of the provided category options in detail. Consider how well each one aligns with the specifics of the product based on the title. Consider the hierarchy category structure.
    3. If the title explicitly mentions a specific material, style, size, or feature that corresponds to a more specific leaf category, prioritize that category over the more general one.
    4. If none of the category options seem to fit the product well, still select the closest match rather than a category not on the list. 
    5. Return the leaf_category of your selected category in the JSON format with key, leaf_category.

Remember:
Select a category only from the provided options, even if it's not a perfect fit. Do not generate new categories.


Examples:
<user>: "product title:Plus Size Women'S Backless Halter Neck Belted Denim Dress.\n category choices:[{'sku_cate_1_nm': 'Women Apparel', 'sku_cate_2_nm': 'Women Plus Clothing', 'sku_cate_3_nm': 'Plus Size Denim', 'sku_cate_4_nm': 'Plus Size Denim Dresses'}, {'sku_cate_1_nm': 'Women Apparel', 'sku_cate_2_nm': 'Women Plus Clothing', 'sku_cate_3_nm': 'Plus Size Denim', 'sku_cate_4_nm': 'Plus Size Denim Two-piece Outfits'}]"
<assistant>: {"leaf_category": "Plus Size Denim Dresses"}

<user>: "product title:2pcs/Set Lace Cloth Water Dispenser Cover Dust-Proof Bucket Cover For Home Living Room Modern & Simple Style.\n category choices:[{'sku_cate_1_nm': 'Home & Living', 'sku_cate_2_nm': 'Household Merchandises', 'sku_cate_3_nm': 'Dust Covers'},  {'sku_cate_1_nm': 'Home & Living', 'sku_cate_2_nm': 'Storage & Organization', 'sku_cate_3_nm': 'Laundry Storage & Organization', 'sku_cate_4_nm': 'Laundry Buckets'}]"
<assistant>: {"leaf_category": "Dust Covers"}

"""

print(prompt_select_v2)


Your task is to choose the most appropriate category for a given product title from the provided list of category choices. 

Steps:

    1. Analyze the product title to understand the product.
    2. Review each of the provided category options in detail. Consider how well each one aligns with the specifics of the product based on the title. Consider the hierarchy category structure.
    3. If the title explicitly mentions a specific material, style, size, or feature that corresponds to a more specific leaf category, prioritize that category over the more general one.
    4. If none of the category options seem to fit the product well, still select the closest match rather than a category not on the list. 
    5. Return the leaf_category of your selected category in the JSON format with key, leaf_category.

Remember:
Select a category only from the provided options, even if it's not a perfect fit. Do not generate new categories.


Examples:
<user>: "product title:Plus Size Women'S Bac

In [ ]:
dataframe_request_notna(df_prod, 'input', prompt_select_v2, temp=0.2, num_threads=1, model='chat')

In [ ]:
#check completeness

df_extract_category_type(['leaf_category'], df_prod, 'api_response')
print(df_prod['leaf_category'].value_counts(dropna=False))
print(len(df_prod))
print(len(df_prod[df_prod['api_response']=='']))
print(len(df_prod[df_prod['api_response'].isna()]))
print(len(df_prod[df_prod['api_response']=='None']))

In [344]:
# sample result df_prod_test = pd.read_excel('./df_prod_samp_emb_top_50_leaf_0413.xlsx')

print("Groundtruth: \n", df_prod_test[['sku_cate_1_nm','sku_cate_2_nm','sku_cate_3_nm','sku_cate_4_nm']].head(1))
print('\n')
print("Result: \n", df_prod_test['leaf_category'][0])

print('\n')
print("Candidate: \n", df_prod_test['top_similar_json'][0])

df_prod_test[['goods_name_en','sku_cate_1_nm','sku_cate_2_nm','sku_cate_3_nm','sku_cate_4_nm','top_similar_json','input','api_response','leaf_category']].head(3)

Groundtruth: 
    sku_cate_1_nm        sku_cate_2_nm    sku_cate_3_nm  \
0  Women Apparel  Women Plus Clothing  Plus Size Denim   

             sku_cate_4_nm  
0  Plus Size Denim Dresses  


Result: 
 Plus Size Denim Dresses


Candidate: 
 [{'sku_cate_1_nm': 'Women Apparel', 'sku_cate_2_nm': 'Women Plus Clothing', 'sku_cate_3_nm': 'Plus Size Denim', 'sku_cate_4_nm': 'Plus Size Denim Dresses'}, {'sku_cate_1_nm': 'Women Apparel', 'sku_cate_2_nm': 'Women Plus Clothing', 'sku_cate_3_nm': 'Plus Size Denim', 'sku_cate_4_nm': 'Plus Size Denim Two-piece Outfits'}, {'sku_cate_1_nm': 'Women Apparel', 'sku_cate_2_nm': 'Women Plus Clothing', 'sku_cate_3_nm': 'Plus Size Denim', 'sku_cate_4_nm': 'Plus Size Denim Tops'}, {'sku_cate_1_nm': 'Women Apparel', 'sku_cate_2_nm': 'Women Plus Clothing', 'sku_cate_3_nm': 'Women Plus Beachwear', 'sku_cate_4_nm': 'Plus Size Swim Dresses'}, {'sku_cate_1_nm': 'Women Apparel', 'sku_cate_2_nm': 'Weddings & Events', 'sku_cate_3_nm': 'Women Plus Wedding', 'sku_cate_4

,goods_name_en,sku_cate_1_nm,sku_cate_2_nm,sku_cate_3_nm,sku_cate_4_nm,top_similar_json,input,api_response,leaf_category
0,Plus Size Women'S Backless Halter Neck Belted Denim Dress,Women Apparel,Women Plus Clothing,Plus Size Denim,Plus Size Denim Dresses,"[{'sku_cate_1_nm': 'Women Apparel', 'sku_cate_2_nm': 'Women Plus Clothing', 'sku_cate_3_nm': 'Plus Size Denim', 'sku_cate_4_nm': 'Plus Size Denim Dresses'}, {'sku_cate_1_nm': 'Women Apparel', 'sku_cate_2_nm': 'Women Plus Clothing', 'sku_cate_3_nm': 'Plus Size Denim', 'sku_cate_4_nm': 'Plus Size Denim Two-piece Outfits'}, {'sku_cate_1_nm': 'Women Apparel', 'sku_cate_2_nm': 'Women Plus Clothing'...","product title:Plus Size Women'S Backless Halter Neck Belted Denim Dress.\n category choices:[{'sku_cate_1_nm': 'Women Apparel', 'sku_cate_2_nm': 'Women Plus Clothing', 'sku_cate_3_nm': 'Plus Size Denim', 'sku_cate_4_nm': 'Plus Size Denim Dresses'}, {'sku_cate_1_nm': 'Women Apparel', 'sku_cate_2_nm': 'Women Plus Clothing', 'sku_cate_3_nm': 'Plus Size Denim', 'sku_cate_4_nm': 'Plus Size Denim Tw...","{""leaf_category"": ""Plus Size Denim Dresses""}",Plus Size Denim Dresses
1,Plus Size Slim Fit Shorts With Sparkly Patchwork,Women Apparel,Women Plus Clothing,Plus Size Bottoms,Plus Size Shorts,"[{'sku_cate_1_nm': 'Sports & Outdoor', 'sku_cate_2_nm': 'Women Plus Activewear', 'sku_cate_3_nm': 'Plus Size Sports Bottoms', 'sku_cate_4_nm': 'Plus Size Sports Shorts'}, {'sku_cate_1_nm': 'Women Apparel', 'sku_cate_2_nm': 'Women Plus Clothing', 'sku_cate_3_nm': 'Plus Size Bottoms', 'sku_cate_4_nm': 'Plus Size Shorts'}, {'sku_cate_1_nm': 'Women Apparel', 'sku_cate_2_nm': 'Women Plus Clothing',...","product title:Plus Size Slim Fit Shorts With Sparkly Patchwork.\n category choices:[{'sku_cate_1_nm': 'Sports & Outdoor', 'sku_cate_2_nm': 'Women Plus Activewear', 'sku_cate_3_nm': 'Plus Size Sports Bottoms', 'sku_cate_4_nm': 'Plus Size Sports Shorts'}, {'sku_cate_1_nm': 'Women Apparel', 'sku_cate_2_nm': 'Women Plus Clothing', 'sku_cate_3_nm': 'Plus Size Bottoms', 'sku_cate_4_nm': 'Plus Size S...","{""leaf_category"": ""Plus Size Sports Shorts""}",Plus Size Sports Shorts
2,Plus Size Women's Color Block Suit Trousers,Women Apparel,Women Plus Clothing,Plus Size Suits,Plus Size Suit Pants,"[{'sku_cate_1_nm': 'Women Apparel', 'sku_cate_2_nm': 'Women Plus Clothing', 'sku_cate_3_nm': 'Plus Size Suits', 'sku_cate_4_nm': 'Plus Size Suit Pants'}, {'sku_cate_1_nm': 'Women Apparel', 'sku_cate_2_nm': 'Women Plus Clothing', 'sku_cate_3_nm': 'Plus Size Suits', 'sku_cate_4_nm': 'Plus Size Suit Sets'}, {'sku_cate_1_nm': 'Women Apparel', 'sku_cate_2_nm': 'Women Plus Clothing', 'sku_cate_3_nm'...","product title:Plus Size Women's Color Block Suit Trousers.\n category choices:[{'sku_cate_1_nm': 'Women Apparel', 'sku_cate_2_nm': 'Women Plus Clothing', 'sku_cate_3_nm': 'Plus Size Suits', 'sku_cate_4_nm': 'Plus Size Suit Pants'}, {'sku_cate_1_nm': 'Women Apparel', 'sku_cate_2_nm': 'Women Plus Clothing', 'sku_cate_3_nm': 'Plus Size Suits', 'sku_cate_4_nm': 'Plus Size Suit Sets'}, {'sku_cate_1...","{""leaf_category"": ""Plus Size Suit Pants""}",Plus Size Suit Pants


## Result Analysis

In [355]:
#return leaf category
#df_category = pd.read_excel('./category_0322.xlsx')
#df_prod = df_prod_test
leaf_category_set = set(df_category['sku_cate_nm'])


#hallucination
df_prod['not_hall'] = df_prod['leaf_category'].isin(leaf_category_set)

#compute acc
df_prod['acc'] =  df_prod['leaf_category'] == df_prod['cate_last']

#groupby result
print('Not Hallucination')

print(df_prod.groupby(['sku_cate_1_nm']).agg({'not_hall':['sum','count',lambda x: round((x.sum() / x.count()) * 100, 2)]}))

print('\n')

print('Accurary')

print(df_prod.groupby(['sku_cate_1_nm']).agg({'acc':['sum','count',lambda x: round((x.sum() / x.count()) * 100, 2)]}))

print('\n')

print('Accuracy Except Hallucination')
print(df_prod[df_prod['not_hall']==True].groupby(['sku_cate_1_nm']).agg({'acc':['sum','count',lambda x: round((x.sum() / x.count()) * 100, 2)]}))


Not Hallucination
              not_hall                 
                   sum count <lambda_0>
sku_cate_1_nm                          
Home & Living       80   112      71.43
Women Apparel       63    72      87.50


Accurary
              acc                 
              sum count <lambda_0>
sku_cate_1_nm                     
Home & Living  43   112      38.39
Women Apparel  38    72      52.78


Accuracy Except Hallucination
              acc                 
              sum count <lambda_0>
sku_cate_1_nm                     
Home & Living  43    80      53.75
Women Apparel  38    63      60.32
